In [74]:
import spacy
nlp = spacy.load("en_core_web_sm")

def is_simile(sentence, subsentence):
    doc = nlp(sentence)

    # Iterate over the tokens in the sentence
    for token in doc:
        # Check if token is 'like' and is used as a comparison term
        if (token.text == 'like' and token.dep_ == 'prep'):
            # Get the comparison target
            comparison_targets = []
            for child in token.children:
                comparison_targets.append(child.text + child.whitespace_)
            comparison_target = "".join(comparison_targets).strip()
            # Check if the comparison target is in the subsentence
            if comparison_target in subsentence:
                return True

    return False

# Test the function
sentence = "When it comes to cooking, it’s like the blind leading the blind when my brother and I try to help each other"
subsentence = "the blind leading the blind"
print(is_simile(sentence, subsentence)) # True!

True


In [75]:
import pandas as pd
df = pd.read_csv("dataset_translation_ready.csv")
# Creating a new column 'text_without_brackets' by removing the square brackets
df['sentence'] = df['source'].str.replace(r'\[\[', '', regex=True).str.replace(r'\]\]', '', regex=True)

# Creating a new column 'text_inside_brackets' by extracting the text inside the square brackets
df['subsentence'] = df['source'].str.extract(r'\[\[(.*?)\]\]', expand=False)
len(df)

1060

In [76]:
def f(row):
    row['is_simile'] = is_simile(row['sentence'], row['subsentence'])
    return row
df = df.apply(f, axis=1)

In [77]:
df[df['is_simile']][['idiom', 'meaning', 'source', 'subsentence']].to_csv('is_simile.csv', index=False)

In [78]:
import pandas as pd
import numpy as np

# Let's assume that df is your DataFrame and "is_simile" is one of its columns
# Create a new column 'is_simile_pair', initialized with False
df['is_simile_pair'] = False

# Loop through the DataFrame with step 2
for i in range(0, len(df), 2):
    # Set 'is_simile_pair' as True if 'is_simile' is True for any of the two rows
    df.loc[i, 'is_simile_pair'] = df.loc[i, 'is_simile'] or df.loc[i+1, 'is_simile']
    df.loc[i+1, 'is_simile_pair'] = df.loc[i, 'is_simile'] or df.loc[i+1, 'is_simile']

df = df[['idiom', 'meaning', 'label', 'source', 'is_simile_pair']]

In [79]:
df.to_csv("to_be_translated.csv", index=False)